In [2]:
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import spacy

OSError: [WinError 1455] 页面文件太小，无法完成操作。 Error loading "d:\python38\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.

In [ ]:
print(torch.__version__)

In [ ]:
torch.cuda.is_available()


# About the version of package:

1. If you don't want to mass you package version, you can create the virtual environment
only for this project.
please see the detail: https://docs.python.org/3.8/library/venv.html

2. After you create virtual environment, if you use pip, you can install pipreqs package,
it can help you to save you package version and easy to install:

pip install -r .\requirements.txt
please see the detail：https://github.com/bndr/pipreqs

I will provide my package version for you in the file.


3. For this project, The most important is torchtext 0.9, all the minor version is works,
torch 1.8.1 and torchtext 0.9.1 is also work.
If you not use GPU, you can just use torch 1.8.1, which is same as Professor Li publish in blackboard.
If you want to use GPU, please check torch version: https://pytorch.org/get-started/locally/
and torchtext version: https://pypi.org/project/torchtext/

If you have some questions, please contact me.

# The data set is from kaggle youtube  statistics:
https://www.kaggle.com/code/elem3ntary/predicting-comment-sentiment/data

The code is from :https://sebastianraschka.com/blog/2021/dl-course.html

# Comment Sentiment:
0 represents a negative sentiment
1 represent neutral
2 represent positive

In [ ]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.01
BATCH_SIZE = 128
NUM_EPOCHS = 100
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_CLASSES = 3

In [ ]:
print(DEVICE)

In [ ]:
# read the data
comment =  pd.read_csv('comments.csv')
comment.groupby('Sentiment').count()['Comment'].plot.bar()


In [ ]:
#look at data
comment.head()

In [ ]:
comment.iloc[3,2]

In [ ]:
# look the data
comment.describe()

In [ ]:
#have 1 null data in comment
comment.isnull().sum()

In [ ]:
#look at null data
comment[comment['Comment'].isnull()]


In [ ]:
#drop the other columns, and change 1 and 2 in sentiment to 1
comment_data = comment.drop(columns = ['Unnamed: 0', 'Video ID', 'Likes'])
comment_data.groupby('Sentiment').count()

In [ ]:
#change column name
comment_data.columns = ['TEXT_COLUMN_NAME', 'LABEL_COLUMN_NAME']

In [ ]:
comment_data.to_csv('youtube_data.csv', index=None)
comment_look = pd.read_csv('youtube_data.csv')



Download English vocabulary via:

- `python -m spacy download en_core_web_sm`



In [ ]:
# only have legacy in torchtext==0.9
# pip install torchtext==0.9
### Defining the feature processing

TEXT = torchtext.legacy.data.Field(
    tokenize='spacy', # default splits on whitespace
    tokenizer_language='en_core_web_sm'
)

### Defining the label processing

LABEL = torchtext.legacy.data.LabelField(dtype=torch.long)


In [ ]:
fields = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]

dataset = torchtext.legacy.data.TabularDataset(
    path='youtube_data.csv', format='csv',
    skip_header=True, fields=fields)



In [ ]:
# separate into train and test set
train_data, test_data = dataset.split(
    split_ratio=[0.8, 0.2],
    random_state=random.seed(RANDOM_SEED))

print(f'Num Train: {len(train_data)}')
print(f'Num Test: {len(test_data)}')


In [ ]:
# separate into train, valid and test set
train_data, valid_data = train_data.split(
    split_ratio=[0.85, 0.15],
    random_state=random.seed(RANDOM_SEED))

print(f'Num Train: {len(train_data)}')
print(f'Num Validation: {len(valid_data)}')


In [ ]:
print(vars(train_data.examples[0]))


## Build Vocabulary

Build the vocabulary based on the top "VOCABULARY_SIZE" words:

In [ ]:
TEXT.build_vocab(train_data, max_size=VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

- 20,002 not 20,000 because of the `<unk>` and `<pad>` tokens
- PyTorch RNNs can deal with arbitrary lengths due to dynamic graphs, but padding is necessary for padding sequences to the same length in a given minibatch so we can store those in an array

In [ ]:
#Look at most common words:
print(TEXT.vocab.freqs.most_common(20))


In [ ]:
#Tokens corresponding to the first 10 indices (0, 1, ..., 9):
print(TEXT.vocab.itos[:10]) # itos = integer-to-string



**Converting a string to an integer:**

In [ ]:
print(TEXT.vocab.stoi['the']) # stoi = string-to-integer

**Class labels:**

In [ ]:
print(LABEL.vocab.stoi)

**Class label count:**

In [ ]:
LABEL.vocab.freqs

## Define Data Loaders

In [ ]:
train_loader, valid_loader, test_loader = \
    torchtext.legacy.data.BucketIterator.splits(
        (train_data, valid_data, test_data),
         batch_size=BATCH_SIZE,
         sort_within_batch=False,
         sort_key=lambda x: len(x.TEXT_COLUMN_NAME),
         device=DEVICE
    )

Testing the iterators (note that the number of rows depends on the longest document in the respective batch):

In [ ]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

## Model

In [ ]:
class RNN(torch.nn.Module):

    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        #self.rnn = torch.nn.RNN(embedding_dim,
        #                        hidden_dim,
        #                        nonlinearity='relu')
        self.rnn = torch.nn.LSTM(embedding_dim,
                                 hidden_dim)

        self.fc = torch.nn.Linear(hidden_dim, output_dim)


    def forward(self, text):
        # text dim: [sentence length, batch size]

        embedded = self.embedding(text)
        # embedded dim: [sentence length, batch size, embedding dim]

        output, (hidden, cell) = self.rnn(embedded)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]

        hidden.squeeze_(0)
        # hidden dim: [batch size, hidden dim]

        output = self.fc(hidden)
        return output

In [ ]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim=len(TEXT.vocab),
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=NUM_CLASSES # could use 1 for binary classification
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

## Training

In [ ]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100


In [ ]:
def predict_label(model, data_loader, device):

    with torch.no_grad():
        Y_acctual, Y_preds = [], []
        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
            Y_preds.append(predicted_labels.cpu().data.numpy())
            Y_acctual.append(targets.cpu().data.numpy())
    return  np.concatenate(Y_preds), np.concatenate(Y_acctual)

In [ ]:
start_time = time.time()

train_accuracy = []
valid_accuracy = []

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):

        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)

        ### FORWARD AND BACK PROP
        logits = model(text)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()

        loss.backward()

        ### UPDATE MODEL PARAMETERS
        optimizer.step()

        ### LOGGING
        if not batch_idx % 50:

            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        train_accuracy.append(float(f'{compute_accuracy(model, train_loader, DEVICE):.2f}'))
        valid_accuracy.append(float(f'{compute_accuracy(model, valid_loader, DEVICE):.2f}'))
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')

    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')

print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')


In [ ]:


nlp = spacy.blank("en")

def predict_sentiment(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    return prediction[0][0].item()

In [ ]:
plt.plot(train_accuracy, label = 'train')
plt.plot(valid_accuracy, label = 'valid')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.savefig('accuracy.png')

In [ ]:
print('Probability positive:')
predict_sentiment(model, "This is such an awesome movie, I really love it!")


In [ ]:
print('Probability negative:')
1-predict_sentiment(model, "Whenever I go to a place that doesn’t take Apple Pay (doesn’t happen too often), it’s such a drag. Between ‘contactless Covid’ habits and my getting the Apple Card, I’ve gotten so used to Apple Pay that I get seriously annoyed when a store doesn’t take it. It feels like a shock, it’s crazy how quickly it took over my shopping routine! I’ve officially been brainwashed by Apple because now it feels so inconvenient to even carry a physical card in my pocket")

In [ ]:
predict_label, real_label= predict_label(model, test_loader, DEVICE)
print(predict_label, real_label)

In [ ]:
#confuion matrix

cf_matrix = confusion_matrix(real_label, predict_label)
cf_matrix

In [ ]:
ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt = '.20g')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('Predicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['2','1','0'])
ax.yaxis.set_ticklabels(['2','1','0'])

plt.savefig('confusion_matrix.png')
## Display the visualization of the Confusion Matrix.
plt.show()